In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

# 데이터셋 분할 방법 - Hold out
### Iris 데이터셋 사용

In [16]:
X, y = load_iris(return_X_y=True) # return_X_y = True : input(X), output(y) 값만 조회해서 가져온다
X.shape, y.shape

((150, 4), (150,))

In [17]:
X_tmp, X_test, y_tmp, y_test = train_test_split(X, # input (feature, X)
                                                y, # output (target, label, y)
                                                test_size=0.2, # (test set의 크기),  
                                                               # 0~1 실수 : 비율 / 정수 : 개수로 나뉨
                                                               # default 0.25
                                                stratify=y, # 분류에서 필수, 원본 데이터셋의 클래스 별 비율을 
                                                                               # 유지하면서 나눠지도록 처리.
                                                
                                                random_state=0 # random.seed 값
                                               )

X_tmp.shape, X_test.shape, y_tmp.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.2, 
                                                  stratify=y_tmp, random_state=0)

In [19]:
X_train.shape, X_val.shape, X_test.shape

((96, 4), (24, 4), (30, 4))

In [20]:
# max_depth = 1
# max_depth = 2
# max_depth = 3
max_depth = 5

# max_depth를 바꿔가면서 하나씩 하면서 확인

# max_depth : DecisionTree 의 하이퍼 파라미터 중 하나. 사람이 직접 설정한 파라미터 값.
tree = DecisionTreeClassifier(max_depth=max_depth, random_state=0)
tree.fit(X_train, y_train) # train set 으로 학습

# 검증
## 1. 추정
pred_train = tree.predict(X_train) # train set feature를 입력해서 추정
pred_val = tree.predict(X_val) # validation set feature를 입력해서 추정

## 2. 검증
train_acc = accuracy_score(y_train, pred_train) # (정답, 모델 추정값)
val_acc = accuracy_score(y_val, pred_val) # (정답, 모델 추정값)

🚩 `max_depth`가 1인 상황 : `train set`의 정확도가 0.6666666, 학습했던 데이터셋에 대한 정확도를 검증했는데도 성능이 너무 안나온 상황 -> 애초에 모델에 학습 데이터가 잘 학습되지 않았다는 의미

In [21]:
print(f"max_depth: {max_depth}")
print("train set 정확도:", train_acc)
print("val set 정확도:", val_acc)

max_depth: 5
train set 정확도: 1.0
val set 정확도: 1.0


In [22]:
pred_test = tree.predict(X_test)
test_acc = accuracy_score(y_test, pred_test)
print('test set 정확도:', test_acc)

test set 정확도: 0.9666666666666667


# 데이터셋 분할 방법 - K-Fold Cross Validation

### Boston housing 데이터셋 사용

In [23]:
# from sklearn.datasets import boston
import pandas as pd

boston = pd.read_csv('data/boston_hosing.csv')
boston.shape

(506, 14)

In [24]:
# MEDV -> 해당 데이터뎃의 target(label, y)
# 데이터셋에서 X,y를 분리 후 ndarray로 변환
X = boston.drop(columns='MEDV').values
y = boston['MEDV'].values

# Series and DataFrame '.values' => 'ndarray' 타입으로 반환

In [25]:
X.shape, y.shape

((506, 13), (506,))

## KFold

In [26]:
##################### KFold를 이용해 Boston housing dataset 교차검증
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# fold(iteration) 별 검증 결과를 담을 리스트
mse_list = []

#1. KFold 객체 생성 - 폴드개수 (K) 를 설정
kfold = KFold(n_splits=5)

#2. split을 이용해서 generator 생성
index_gen = kfold.split(X) # 5분할한 데이터셋의 인덱스

#3. 반복문 -> 각 folder별 iteration을 처리.
tree = DecisionTreeRegressor(random_state=0) # DecisionTree 회귀 모델

for train_index, val_index in index_gen:
    # train/val dataset을 생성
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]
    
    # 모델 학습
    tree.fit(X_train, y_train)
    
    # 검증
    ## 추론
    pred_val = tree.predict(X_val)
    
    ## 검증 - mse
    mse = mean_squared_error(y_val, pred_val) # (정답, 모델 추정값)
# mean_squared_error
    mse_list.append(mse)

In [27]:
mse_list

[11.887843137254906,
 34.88990099009901,
 28.17247524752476,
 54.44178217821782,
 52.59029702970297]

In [28]:
np.mean(mse_list), np.sqrt(np.mean(mse_list)) # sqrt : 제곱근

(36.39645971655989, 6.032947846331832)

🚩 회귀 문제는 무한대의 값들을 추정하는 과정이기 때문에 완전히 그 값을 맞춘다는 것은 불가능하다.
따라서 이러한 회귀 문제의 모델 성능 평가는 '얼마나 많이 맞추는가' 가 아니라 **'얼마나 적게 틀렸는가'** 를 
중점적으로 봐야한다. 
따라서 사용되는 메서드도 분류문제의 `accuracy_score`가 아닌 **`mean_squared_error`** 를 사용해서 모델 성능 평가를 한다.

In [29]:
# 서비스하기 위한 모델은 전체 데이터셋을 가지고 다시 학습시킨다.
tree.fit(X, y)

DecisionTreeRegressor(random_state=0)

## StratifiedKFold

In [30]:
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [31]:
X, y = load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

In [32]:
############# iris dataset 교차 검증

# iteration별 평가결과(정확도)를 저장할 리스트
acc_list = []

#1. KFold 객체 생성 - 폴드개수 (K) 를 설정
s_kfold = StratifiedKFold(n_splits=4)

#2. split을 이용해서 generator 생성
index_gen_iris = s_kfold.split(X, y)

#3. 반복문 -> 각 folder별 iteration을 처리.
tree_clf = DecisionTreeClassifier(random_state=0) # 분류 문제이기 때문에 Classifier, 학습할 모델

for train_idx, val_idx in index_gen_iris:
    # train/val dataset을 생성
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]
    
    #  모델학습
    tree_clf.fit(X_train, y_train)
    
    # 검증
    ## 추론
    pred = tree_clf.predict(X_val)
    acc = accuracy_score(y_val, pred)
    
    ## 검증
    acc_list.append(acc)

In [33]:
acc_list

[0.9736842105263158, 0.9473684210526315, 0.9459459459459459, 1.0]

In [34]:
np.mean(acc_list)

0.9667496443812233

In [38]:
# 서비스하기 위한 모델은 전체 데이터셋을 가지고 다시 학습시킨다.
tree_clf.fit(X, y)

DecisionTreeClassifier(random_state=0)

Why? **분류 문제**에서 `StratifiedKFold`를 사용하고<br> `KFold`는 사용하지 않는 지 위에서 한 데이터셋 그대로 `KFold`로 해보자

In [41]:
# KFold 로 왜 이걸로 안하는 지 알아보자
kf = KFold(n_splits=3)
g = kf.split(X)
i1, i2 = next(g)
print(f"y[i1] : {y[i1]}")
print(f"y[i2] : {y[i2]}")

# train과 test data set 의 y 값이 극과 극으로 나누어져 있어 모델 성능 평가의 신뢰성이 없기 때문이다.
# 위의 코드 대로 하면 y[i1] 은 1과 2로만 이루어져 있고, y[i2]는 0 으로만 이루어져 있기 때문에 
# 학습한 모델을 평가하는데 의미가 없어졌습니다.

y[i1] : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
y[i2] : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


> `train`과 `test data set` 의 `y` 값이 극과 극으로 나누어져 있어 모델 성능 평가의 신뢰성이 없기 때문이다.
위의 코드 대로 하면 `y[i1]` 은 **1과 2로만** 이루어져 있고, `y[i2]`는 **0 으로만** 이루어져 있기 때문에 
**학습한 모델을 평가하는데 의미가 없어졌습니다.**

# Cross Validation Utility 함수

In [42]:
# 보스톤데이터셋 - 회귀
df = pd.read_csv('data/boston_hosing.csv')
X_boston = df.drop(columns='MEDV').values
y_boston = df['MEDV'].values

# iris - 분류
X_iris, y_iris = load_iris(return_X_y=True)

In [43]:
from sklearn.model_selection import cross_val_score, cross_validate

result_boston = cross_val_score(DecisionTreeRegressor(random_state=0, max_depth=3), # 학습 모델
                                X=X_boston, # features, X, input
                                y=y_boston, # label, target, y, output
                                
                                scoring="neg_mean_squared_error", # 평가지표. -mse
                                
                                cv= 4, # fold 개수 == KFold(n_splits=4)
                                n_jobs= -1 # 병렬처리할 때 사용할 cpu의 프로세서 개수. default == 1
                                        # -1 : 모든 프로세서 다 사용한다.
)

# 위에 만든 cv 함수를 cross_val_score() 메서드가 대신 한다.

In [44]:
print(result_boston)
print(-result_boston)
print(-np.mean(result_boston))

[-19.8971664  -23.5349172  -51.40872605 -31.38321557]
[19.8971664  23.5349172  51.40872605 31.38321557]
31.556006304145065


In [45]:
result_iris = cross_val_score(DecisionTreeClassifier(max_depth=3, random_state=0), 
                              X=X_iris, 
                              y=y_iris, 
                              scoring="accuracy", 
                              cv=4,
                              n_jobs=-1)

In [46]:
print(result_iris)
print(np.mean(result_iris))

[0.97368421 0.94736842 0.94594595 0.97297297]
0.9599928876244666


In [47]:
# 여러 평가지표를 확인
result = cross_validate(DecisionTreeRegressor(random_state=0, max_depth=3), 
                        X=X_boston, 
                        y=y_boston,
                        scoring=['r2', 'neg_mean_squared_error'], 
                        cv=4, 
                        n_jobs=-1                        
                       )

In [48]:
result # 딕셔너리

{'fit_time': array([0.00199461, 0.00398898, 0.00199461, 0.00199437]),
 'score_time': array([0.00099659, 0.00199509, 0.00099659, 0.        ]),
 'test_r2': array([ 0.3224128 ,  0.76128228,  0.4275699 , -0.08225262]),
 'test_neg_mean_squared_error': array([-19.8971664 , -23.5349172 , -51.40872605, -31.38321557])}

In [49]:
result['test_neg_mean_squared_error']

array([-19.8971664 , -23.5349172 , -51.40872605, -31.38321557])

### `neg_mean_squared_error` :
- neg -> negative : `neg_mean_squared_error` 의 값을 **'음수'**로 반환하겠다.
> `mean_squared_error`는 오차의 값이기 때문에 절대값이 작을수록 좋다.
정확한 비교를 위해 '음수'로 반환하여 절대값의 크기가 작을수록
좋은 평가임을 수학적으로 표현하기 위해서 **"neg_"** 를 부여하였다.

# 데이터 전처리
## 결측치 처리

In [50]:
import pandas as pd
import numpy as np

# 결측치 처리 실행 예시 데이터
data = {
    "col1":[10, np.nan, 30, 40], # 결측치는 float 타입이므로, 다른 정수들도 float 타입으로 입력된다.
    "col2":['A', 'A', 'C', np.nan],
    "col3":[10.5, 2.8, np.nan, 9.7]
}
df = pd.DataFrame(data)
df

,col1,col2,col3
0,10.0,A,10.5
1,NaN,A,2.8
2,30.0,C,NaN
3,40.0,NaN,9.7


In [51]:
df.isna().sum()

col1    1
col2    1
col3    1
dtype: int64

In [52]:
df.isna().sum(axis=1)

0    0
1    1
2    1
3    1
dtype: int64

In [53]:
df.dropna()

,col1,col2,col3
0,10.0,A,10.5


In [54]:
df.dropna(subset=['col1', 'col3'])

,col1,col2,col3
0,10.0,A,10.5
3,40.0,NaN,9.7


In [57]:
df.dropna(axis=1) # 결측치가 존재하는 열 삭제, index만 살아남은 모습

""
0
1
2
3


In [58]:
df.dropna(axis=1, subset=[1, 2])

,col2
0,A
1,A
2,C
3,NaN


In [59]:
#### 변경
df.fillna(10)

# 해당 컬럼의 다른 값들의 데이터 타입에 맞춰서 더 큰 개념의 데이터 타입으로 통일 시킨다.

,col1,col2,col3
0,10.0,A,10.5
1,10.0,A,2.8
2,30.0,C,10.0
3,40.0,10,9.7


In [60]:
df.fillna({
    "col1":1000,
    "col2":"가"
})

,col1,col2,col3
0,10.0,A,10.5
1,1000.0,A,2.8
2,30.0,C,NaN
3,40.0,가,9.7


In [61]:
df[['col1', 'col3']].median()

col1    30.0
col3     9.7
dtype: float64

In [62]:
df['col2'].mode() # 최빈값 -> A

0    A
Name: col2, dtype: object

In [63]:
#### scikit-learn을 이용해서 결측치를 다른 값으로 대체하는 방법 ####
from sklearn.impute import SimpleImputer
imputer_con = SimpleImputer(strategy="median")
imputer_cate = SimpleImputer(strategy="most_frequent")

# 연속형
# imputer_con.fit(df[['col1', 'col3']]) # 어떻게 바꿀지 학습 - 컬럼별로 중앙값 계산하는 과정
# r1 = imputer_con.transform(df[['col1', 'col3']]) # 변경
r1 = imputer_con.fit_transform(df[['col1', 'col3']]) # 위의 두 fit, transform 과정을 나누지 않고 한 번에 해결가능.
r1

array([[10. , 10.5],
       [30. ,  2.8],
       [30. ,  9.7],
       [40. ,  9.7]])

In [64]:
# 범주형
# fit() : 학습, 최빈값 찾기 / transform() : 찾은 최빈값으로 결측치 대체
r2 = imputer_cate.fit_transform(df[['col2']])
r2

array([['A'],
       ['A'],
       ['C'],
       ['A']], dtype=object)

In [65]:
# 연속형, 범주형으로 나눠서 결측치를 대체한 후
# 결과를 다시 하나로 합치는 과정.
result = np.concatenate([r1, r2], axis=1)
result

array([[10.0, 10.5, 'A'],
       [30.0, 2.8, 'A'],
       [30.0, 9.7, 'C'],
       [40.0, 9.7, 'A']], dtype=object)